In [1]:
import jax
import jax.numpy as jnp
from crystalformer.src.utils import GLXYZAW_from_file
from crystalformer.src.elements import element_dict
from crystalformer.src.formula import find_composition_vector, formula_string
from crystalformer.src.wyckoff import mult_table

ERROR:2025-10-21 17:51:21,170:jax._src.xla_bridge:487: Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/home/user_wanglei/.local/lib/python3.13/site-packages/jax/_src/xla_bridge.py", line 485, in discover_pjrt_plugins
    plugin_module.initialize()
    ~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/home/user_wanglei/.local/lib/python3.13/site-packages/jax_plugins/xla_cuda12/__init__.py", line 328, in initialize
    _check_cuda_versions(raise_on_first_error=True)
    ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user_wanglei/.local/lib/python3.13/site-packages/jax_plugins/xla_cuda12/__init__.py", line 285, in _check_cuda_versions
    local_device_count = cuda_versions.cuda_device_count()
RuntimeError: jaxlib/cuda/versions_helpers.cc:113: operation cuInit(0) failed: Unknown CUDA error 303; cuGetErrorName failed. This probably means that JAX was unable to load the CUDA libraries.


In [2]:
#alex20_folder = '/opt/data/bcmdata/ZONES/data/PROJECTS/datafile/PRIVATE/zdcao/crystal_gpt/dataset/alex/PBE/alex20/'
alex20_folder = '/opt/data/bcmdata/ZONES/data/PROJECTS/datafile/PRIVATE/zdcao/crystal_gpt/dataset/alex/PBE_20241204/'

train_path = alex20_folder+'/train.lmdb'
valid_path = alex20_folder+'/val.lmdb'
test_path = alex20_folder+'/test.lmdb'


In [3]:
atom_types = 119 
wyck_types = 28 
n_max = 21

train_dataset = GLXYZAW_from_file(train_path, atom_types, wyck_types, n_max)
valid_dataset = GLXYZAW_from_file(valid_path, atom_types, wyck_types, n_max)
test_dataset = GLXYZAW_from_file(test_path, atom_types, wyck_types, n_max)





G: (1387800,)
L: (1387800, 6)
XYZ: (1387800, 21, 3)
A: (1387800, 21)
W: (1387800, 21)
G: (173475,)
L: (173475, 6)
XYZ: (173475, 21, 3)
A: (173475, 21)
W: (173475, 21)
G: (173475,)
L: (173475, 6)
XYZ: (173475, 21, 3)
A: (173475, 21)
W: (173475, 21)


In [4]:
G, L, XYZ, A, W = train_dataset

In [5]:
def find_element(x, elements):
    allowed_values = [0] + [element_dict[e] for e in elements]
    allowed = jnp.asarray(allowed_values)

    # eq[i, j, k] = (x[i, j] == allowed[k])
    eq = (x[:, :, None] == allowed[None, None, :])

    # (1) only allowed values: for each element, it's equal to at least one allowed
    only_allowed = jnp.all(jnp.any(eq, axis=2), axis=1)   # shape (n,)

    # (2) all allowed present: for each allowed value k, it appears in the row at least once
    all_present = jnp.all(jnp.any(eq, axis=1), axis=1)    # shape (n,)

    return jnp.where(only_allowed & all_present)[0]

In [24]:
idx = find_element(A, ['Y', 'In', 'Mn'])


In [25]:
A[idx]

Array([], shape=(0, 21), dtype=int32)

In [26]:
G[idx]

Array([], shape=(0,), dtype=int32)

In [27]:
@jax.vmap
def lookup(g, w):
    return mult_table[g-1, w]
M = lookup(G[idx], W[idx]) # (batchsize, n_max)


In [17]:
composition = jax.vmap(find_composition_vector)(A[idx], M)

In [18]:
for c in composition:
    print (formula_string(c))

Ni5La9O19
NiLaO3
Ni4La8O17
NiLa2O4
Ni8La16O33
NiLaO3
NiLaO3
Ni2La3O7
NiLa2O4
NiLaO3
NiLa4O7


In [12]:
W[idx][26]

Array([1, 1, 2, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],      dtype=int32)

In [14]:
A[idx][26]

Array([3, 3, 3, 5, 6, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],      dtype=int32)

In [15]:
XYZ[idx][26]

Array([[0.        , 0.        , 0.13285412],
       [0.        , 0.        , 0.61676514],
       [0.        , 0.5       , 0.12457615],
       [0.1668224 , 0.25403386, 0.37660232],
       [0.1677413 , 0.7505205 , 0.3744405 ],
       [0.66486776, 0.2470655 , 0.12661682],
       [0.6710119 , 0.7428827 , 0.12443022],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]], dtype=float32)